In [1]:
import os
import csv
from pprint import pprint
import pandas as pd
import pickle
import collections

In [2]:
hashPath = "/Users/Bya/git/EPLdata/"
dataPath = "/Users/Bya/Dropbox/Research/datas/"
modelPath = "/Users/Bya/Dropbox/Research/datas/Pickles/Models/"
tokenizerPath = "/Users/Bya/git/predictEPL/Tokenizers/"

os.chdir(tokenizerPath)
import sentiment_aware as sa
os.chdir(hashPath)
from config import Hashtags

In [3]:
model = "MovieShortReview"

# read saved classifier
saved_classifier_f = open(modelPath + model + "/naiveBayes.pickle", "rb")
saved_classifier = pickle.load(saved_classifier_f)
saved_classifier_f.close()

# read saved word features
word_features5k_f = open(modelPath + model + "/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()

# Define Sentiment Tokenizer Class as tok
tok = sa.Tokenizer(preserve_case=False)
def find_features(document):
    words = tok.tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Define Sentiment Analysis function. Using aboves
def sentiment(text):
    return saved_classifier.classify(find_features(text))

In [4]:
def whichSide(tags, tags_home, tags_away):
    tags = map(lambda tag: '#' + tag.lower(), tags)
    if set(tags).intersection(tags_home) and set(tags).intersection(tags_away):
        return 'both'
    elif set(tags).intersection(tags_home):
        return 'home'
    elif set(tags).intersection(tags_away):
        return 'away'
    else:
        return 'nothing'

In [5]:
def ReadFileAsDF(fileName, home_team, away_team):
    # both team hashtags
    tags_home = Hashtags.dic[home_team]
    tags_away = Hashtags.dic[away_team]

    # read file as dataframe, and add 'side' column that shows which team's tweet it is 
    dfTweets = pd.read_csv(fileName, header=None, names=['date', 'text', 'user', 'tags'])
    dfTweets['side'] = map(lambda tags: whichSide(tags, tags_home, tags_away), map(lambda tag: tag.split(','), dfTweets['tags']))
    
    # count tweets
    sides = ['home', 'away', 'both', 'nothing']
    numSides =  map(lambda side: (side, len(dfTweets[dfTweets["side"] == side])), sides)
    print numSides
    
    dfHomeTweets = dfTweets[dfTweets["side"] == 'home']
    dfAwayTweets = dfTweets[dfTweets["side"] == 'away']
    dfHomeTweets = pd.DataFrame(dfHomeTweets.values, range(len(dfHomeTweets)), dfHomeTweets.columns)
    dfAwayTweets = pd.DataFrame(dfAwayTweets.values, range(len(dfAwayTweets)), dfAwayTweets.columns)
    
    return dfHomeTweets, dfAwayTweets

In [6]:
# Create DataFrames for home and away tweets
def SentimentHomeAway(dfHomeTweets, dfAwayTweets):
    home_sent = map(lambda tweet: sentiment(tweet), dfHomeTweets["text"])
    away_sent = map(lambda tweet: sentiment(tweet), dfAwayTweets["text"])
    
    return [collections.Counter(home_sent)['pos'], 
            collections.Counter(home_sent)['neg'],
            collections.Counter(away_sent)['pos'],
            collections.Counter(away_sent)['neg']]

In [7]:
# Save the dictionary values to file
def ListSaveToCSV(fileName, myList):
    with open(fileName + '.csv', 'a') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(myList)

In [8]:
def GameNegPos(game, loadFileName, saveFileName):
    
    home_team = game[0]
    away_team = game[1]

    dfHome, dfAway = ReadFileAsDF(loadFileName, home_team, away_team)
    
    listResSent = SentimentHomeAway(dfHome, dfAway)
    listResGame = [home_team, away_team] + listResSent

    os.chdir("/Users/Bya/Dropbox/Research/datas/Results/")
    ListSaveToCSV(saveFileName, listResGame)

In [10]:
listGamesW6 = [('Chelsea', 'Arsenal'),
               
               ('Villa', 'WestBromwich'),
               ('Bournemouth', 'Sunderland'),
               ('Newcastle', 'Watford'),
               ('Stoke', 'Leicester'),
               ('Swansea', 'Everton'),
               
               ('City', 'WestHam'),
               
               ('Tottenham', 'Crystal'),
               
               ('Liverpool', 'Norwich'),
               ('Southampton', 'United')]

loadFileNameGW6 = ["/Users/Bya/Dropbox/Research/datas/GW6/1509192045_CFCvsARSENAL_json.txt.csv",
                   
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509192300_5matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509192300_5matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509192300_5matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509192300_5matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509192300_5matches_json.txt.csv",

                   "/Users/Bya/Dropbox/Research/datas/GW6/1509200130_MCFCvsWHU_json.txt.csv",
                   
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509202130_COYSvsCPFC_json.txt.csv",
                   
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509210000_2matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW6/1509210000_2matches_json.txt.csv",]

saveFileNameGW6 = 'GW6'

for i in range(len(listGamesW6)):
#     print(listGamesW6[i], loadFileNameGW6[i], saveFileNameGW6)
    GameNegPos(listGamesW6[i], loadFileNameGW6[i], saveFileNameGW6)

[('home', 7120), ('away', 5435), ('both', 6989), ('nothing', 44)]
[('home', 766), ('away', 459), ('both', 1092), ('nothing', 11446)]
[('home', 1065), ('away', 1261), ('both', 858), ('nothing', 10579)]
[('home', 3962), ('away', 1178), ('both', 504), ('nothing', 8119)]
[('home', 506), ('away', 639), ('both', 1254), ('nothing', 11364)]
[('home', 146), ('away', 593), ('both', 396), ('nothing', 12628)]
[('home', 6579), ('away', 4148), ('both', 2062), ('nothing', 24)]
[('home', 2323), ('away', 41), ('both', 623), ('nothing', 2)]
[('home', 7869), ('away', 430), ('both', 2091), ('nothing', 43548)]
[('home', 1018), ('away', 38673), ('both', 3825), ('nothing', 10422)]


In [84]:
# 'Bournemouth'
# 'Arsenal'
# 'Villa'
# 'Chelsea'
# 'Crystal'
# 'Everton'
# 'Leicester'
# 'Liverpool'
# 'City'
# 'United'
# 'Newcastle'
# 'Norwich'
# 'Southampton'
# 'Stoke'
# 'Sunderland'
# 'Swansea'
# 'Tottenham'
# 'Watford'
# 'WestBromwich'
# 'WestHam'